In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import matplotlib.pyplot as plt

# Neural Network Definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 50)  # Input is now 2D: (x_interior, time)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Loss function
def loss_function(u_pred, u_x_pred, u_xxx_pred, u_true, lambda_1, lambda_2):
    f = u_pred[:, 1:] + lambda_1 * u_pred[:, :-1] * u_x_pred[:, :-1] + lambda_2 * u_xxx_pred[:, :-1]
    mse_c = torch.mean(f**2)
    mse_s = torch.mean((u_pred[:, 0] - u_true)**2)
    mse_b = torch.mean((u_pred[:, -1])**2)
    return mse_c + mse_s + mse_b

# Compute first and third derivatives
def derivatives(u, x):
    u_x = autograd.grad(u.sum(), x, create_graph=True)[0]
    u_xx = autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_xxx = autograd.grad(u_xx.sum(), x, create_graph=True)[0]
    return u_x, u_xxx

# Initialize the network and optimizer
net = Net()
optimizer = optim.Adam(net.parameters(), lr=1e-4)
lambda_1 = torch.tensor([1.0], requires_grad=True)
lambda_2 = torch.tensor([1.0], requires_grad=True)

# Data for plotting
epochs = 2000
loss_history = []
lambda_1_history = []
lambda_2_history = []

# Sampling points
x_interior = torch.rand(250, 1, requires_grad=True) * 2 - 1  # Interior points from [-1, 1]
input_t0 = torch.cat((x_interior, t_0), dim=1)
input_t1 = torch.cat((x_interior, t_1), dim=1)

t_0 = torch.full((250, 1), 0.1)
t_1 = torch.full((250, 1), 0.9)
u_true = 2 / torch.cosh(x_interior)

for epoch in range(epochs):
    optimizer.zero_grad()

    # Forward pass for time t_0 and t_1
    u_pred_t0 = net(input_t0)
    u_pred_t1 = net(input_t1)

    # Compute derivatives with respect to x_interior
    u_x_t0, u_xxx_t0 = derivatives(u_pred_t0, x_interior)
    u_x_t1, u_xxx_t1 = derivatives(u_pred_t1, x_interior)

    # Compute loss
    loss = loss_function(u_pred_t0, u_x_t0, u_xxx_t0, u_true, lambda_1, lambda_2)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Store loss and lambda values for plotting
    loss_history.append(loss.item())
    lambda_1_history.append(lambda_1.item())
    lambda_2_history.append(lambda_2.item())

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}, λ1: {lambda_1.item()}, λ2: {lambda_2.item()}')

        # Plot the prediction at the current epoch
        plt.figure(figsize=(8, 5))
        plt.plot(x_interior.detach().numpy(), u_pred_t0[:, 0].detach().numpy(), label=f'u_pred at t_0, epoch {epoch}')
        plt.plot(x_interior.detach().numpy(), u_true.detach().numpy(), label='u_true at t_0', linestyle='dashed')
        plt.title(f'Prediction vs True Solution at Epoch {epoch}')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.legend()
        plt.show()

# Plot the loss over epochs
plt.figure(figsize=(8, 5))
plt.plot(loss_history, label="Training Loss")
plt.title("Loss over Training Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot lambda_1 and lambda_2 over epochs
plt.figure(figsize=(8, 5))
plt.plot(lambda_1_history, label="λ1 over epochs")
plt.plot(lambda_2_history, label="λ2 over epochs")
plt.title("Parameter λ1 and λ2 Convergence")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.legend()
plt.show()


RuntimeError: The size of tensor a (49) must match the size of tensor b (0) at non-singleton dimension 1